# Class 3 - 23.3.20

## Environments

Environments are the Pythonian solution to the "path" problem of MATLAB and other languages.

The basic idea is that every Python project you have has an isolated environment, in which it resides. This environment contains a Python interpreter and libraries that are needed for that specific project, but aren't necessarily required by other projects. 

While it might not seem _that_ important at first, when your project has multiple dependencies on other libraries, asserting compatibility between all components become tricky. 

Python libraries are updated regularly - look at [`numpy`](https://numpy.org/devdocs/release.html), for example, a very important library we'll discuss in a couple of classes. It has a release every few months which fixes bugs, adds new functionality and removes existing ones (after a "deprecation process"). Thousands of libraries depend on `numpy`, but there are so many "relevant" `numpy` versions that someone has to "know" which library uses which version of `numpy`.

This point will be much clearer as the course progresses, so in the mean time just stick with me.

Your different Python environments are managed by a package manager, and the best one (as of early 2020) is `conda`, by Anaconda Inc. (previously Continuum Analytics). conda has two versions, and we'll be working with the slimmer Miniconda distribution (for Python 3.7+), as explained in `PythonSetup.md`.

### Why do we need this?

Last week we saw how Python manages namespaces with the `import` statement. Without importing anything you can't do much with Python, but luckily every Python installation comes with a large __standard library__, i.e. a collection of modules importable with the `import` statement, which can help us do many useful things in our code. We saw examples of `urllib`, `pathlib` and more. Moreover, we saw that using the `pip` tool we can install a countless number of other packages which can help us do basically anything with this language. The big difference between MATLAB and Python in this sense is that external dependencies play a very (very) important role in 99% of Python projects, whilst in MATLAB you rarely depend on files from the MATLAB File Exchange. Thus we need a better system to manage those dependencies in Python.

Moreover, issues will rise when we have multple coding projects, and perhaps multiple users, on the same computer. Each project has its own list of demands (i.e. libraries it uses) and it's very important to make sure that we don't accidently "break" a project, i.e. install a library with the wrong version that will serve two projects. This is why we have environments, which isolate each project with its own interpreter (which is just another library) and set of dependencies.

### How to create an environment?

After you've installed Miniconda and added it to your path, write in the command-line (you can do this from within VSCode as well) `conda create --name my_env python=3.8`. This creates a new Python interpreter named `my_env`. The name should help us associate this environment to a specific project. To make sure we're working with this specific interpreter, we have to activate the environment:
`conda activate my_env`. If you receive a message saying that `conda` is an unrecognized command, please re-install miniconda for "All Users" and also check the "Add to PATH" checkbox, which turns red when you do.

From now on, until we write `conda deactivate`, all Python-related operations will be performed with the `my_env` interpreter in mind. If we have other environments they will not be affected by any change made as long as `my_env` is active. We can also define environment variables which will obviously be specific to this environment alone.

The environment manager we use is called Miniconda, and while Python comes with an environment manager of its own (`virtualenv`), `conda` is the more popular option, especially for scientific Python. `conda` manages dependencies in a more strict fashion than `pip`, and can be used to manage the dependencies of other applications. Miniconda is a "bare-bones" installation, so it doesn't bloat your computer with unneeded packages, allowing for manual control of each package and environment. It also serves for a better learning experience than Anaconda.

For our class you should probably create one environment for the entire semester. While I can't really verify that you're indeed working with environments, I promise you that it's a very useful habit that will save you a great amount of trouble later on, probably without you even noticing it.

![Summary of environments and namespaces](extra_material/envs.png)

## Exercise

- Create a new environment in your computer.
- Create a new folder that will contain a mock Python project, and make this folder a version-controlled one.
- Associate this folder with your new environment in VSCode.
- Add a couple of `.py` files and add to them to the list of tracked files. Make sure to use the right folder structure, as outlined above.
- In one of these `.py` files we'll add a mock function: It should print your name, receiving no input. 
- In the other file, run that function using the `import` statement.
- Install the package `pytz` using `pip` and make sure you're able to `import` it.
- Assuming this project will contain raw `.tif` data, add a `.gitignore` file that disregards the raw data.
- Add an MIT license to the repo, and a basic `README.md`.
- Publish this project to your GitHub account.

# Object-Oriented Programming

## Introduction

There are three main programming paradigms in use in mainstream programming languages:
* Functional
* Procedural
* Object-oriented

While a _functional_ paradigm is very interesting, we'll not be discussing it in this course. You can read about Haskell, OCaml, F# and other functional programming languages wherever you get your information from.

The _procedural_ paradigm is the most widely used paradigm... in the academia. And it's probably the one you're most familiar with from your work in Matlab. 

Confusingly, this paradigm is based around _functions_ (procedures):

In [1]:
# We could write:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
result = []
for item1, item2 in zip(l1, l2):
    result.append(item1 * item2)
result

[4, 10, 18]

In [2]:
# And when we have more lists to multiply we'll again write:
l3 = [10, 20, 30]
l4 = [40, 50, 60]
result2 = []
for item3, item4 in zip(l3, l4):
    result2.append(item3 * item4)
result2

[400, 1000, 1800]

But we're seeing a pattern here. The important DRY principle requires "Don't Repeat Yourself", so we define a function to replace these two implementations:

In [3]:
def list_multiplier(l1, l2):
    """
    Multiply two lists element-wise.
    Returns a list with the result
    """
    result = []
    for item1, item2 in zip(l1, l2):
        result.append(item1 * item2)
    
    return result

This new _procedure_ does one thing, and one thing only. This is what's so powerful about it.

Procedural programming allows us to group and order our code base into small units, called functions or **procedures**, that have a specific, defined task.

It usually contains a "wrapper" script that defines the order of running for these functions:
```python
# my_wrapper_script.py
def run_pipeline(foldername):
    """ Main data pipeline script """
    data = get_user_input(foldername)
    data_without_fieldnames = extract_fieldnames(data)
    columnar_data = generate_columns(data_without_fieldnames, num_of_columns)
    # ...
    # At the end of the file it will contain:
if __name__ == '__main__':
    foldername = r'/path/to/folder'  # raw string
    result = run_pipeline(foldername)
    print(result)
```

You should be extremely decisive and eliminate repeating code. It's perhaps the most common source for errors in scientific computing, and it may bite you any of these ways:

- Encapsulation:
```python
# String concatenation
first_string = 'abcd'
second_string = 'efgh'
concat = first_string + second_string[:-1] + 'zzz'  # you suddenly remember that you wish to exclude 
# the last character in "second_string" and add the 'zzz' sequence at the end.
# Program continues...
# ...
third_string = 'poiu'
fourth_string = 'qwer'
concat2 = third_string + fourth_string + 'zzz' # you wish to achieve the same goal in this 
# concatenation - but you forgot that you excluded the last character of the second string.
```
    The moment you realized that you have a recurring operation on strings - you have to encapsulate it in a function - be _ruthless!_

- Parametrization

```python
def process_data(data):
    scaled_data = data * 0.3  #  what is 0.3 exactly? Parametrize it.
    
def process_data(data, na_concentration=0.3):
    """Multiplies data by the Na concentration"""
    scaled_data = data * na_concentration

```

But this is usually not enough. When calling the `process_data` parameterize the `na_concentration` variable as well:

```python
data = b * c - 1 + a
process_data(data, 0.4)
# Script continues...
process_data(data2, 0.5)  # Perhaps you really did wish to call "process_data" with two different
# parameters, but it's more likely that you decided that 0.5 was too high, so you changed it to 0.4
# in the first call, but forgot that you had a second call. This parameter should appear somewhere at
# the top of your script.
```

## Alternatives

While procedural programming works great most of the time, it can sometime be inferior to other paradigms, namely _object-oriented programming._

But what do I mean by **inferior**? Obviously, all programs can be written successfully without leaving the safe confines of procedural programming.

While this statement holds true, sometimes our _mental model_ of the task at hand fits an object-oriented paradigm more naturally.

### Classes and Objects

Classes are user-defined types. Just like `str`, `dict`, `tuple` and the rest of the standard types, Python allows us to create our own types.

Objects are _instances_ of classes, they're an instance of a type we made. Actually, _all_ instances of _all_ types are objects in Python. It means that every variable and function in Python are, by themselves, an instance of a type. A function you make is an instance of the `function` type, for example. We'll get to this during later stages of the course.

Classes are a type of abstraction we create with our code. A variable is the most simple type of abstraction - it's a _thing_ that is closely tied to a "real value" in a very simple relationship: My variable $x$ represnts the value $y$. 

Classes are more abstract - they don't relate to a specific value directly, but rather they try to convey an idea of an object.

### The Point Class

To show what we mean by "our own type", we'll define the `Point` type.

What is a point? What is composed of?
* In a 2D space it's a pair of values, $(x, y)$, specifying a location on a grid.
* $x$ and $y$ are the coordinates of the point.
* Points have special relations to other points and to the space they reside in.

From these three simple observations, we expect our `Point` type to include both data about its coordinates, and functions, or **methods**, used to interact with the grid and\or other points. 

An object usually bundles together data and methods to use the data.

We wish to express these abstract ideas in our code. It might seem like a lot of code at the start ("boilerplate" code) - but it will be worth it - certainly for more complex classes.

In [4]:
# Introducing the class keyword:
class Point:
    """Represents a point in a 2D space"""

# Notice the capital letter, i.e. CamelCase

In [5]:
Point
# A new type is born in __main__

__main__.Point

In [6]:
# The name Point is now a factory to create new Points.
# To make one, we have to call it like we do with a function:
blank = Point()
blank
# We call this *instantiation*, as blank is an instance of Point

In [7]:
# Assign the point's data in the form of coordinates
blank.x = 1.0
blank.y = 0.0

In [8]:
# x and y are now attributes of our class:
blank.x

1.0

In [ ]:
# import prints_
prints.print_()

The notation `.x` means "go the instance `blank` of the class `Point` and find the value `x` refers to."

There's no conflict between a variable named `x` and the attribute `x`.

The attribute can be used anywhere:

In [9]:
# Simple statements
print(1 + blank.x)

2.0


In [10]:
# Printouts
f"A case of a pointy Point at {(blank.x, blank.y)}"

'A case of a pointy Point at (1.0, 0.0)'

In [11]:
# Arguments to functions
def print_point(p):
    """Print a Point object.
    
    Parameters
    ----------
    p : Point
        The point instance to print
    """
    print(f"{p.x, p.y}")


In [12]:
print_point(blank)

(1.0, 0.0)


### Exercise:
Write a function `distance_between_points(p1, p2)` that takes two points and returns the Cartesian distance between them.

### Exercise solution below...

In [13]:
import math


def distance_between_points(p1, p2):
    """ Calculate Cartesian distance between points """
    return math.sqrt(
        (p1.x - p2.x) ** 2 + 
        (p1.y - p2.y) ** 2
    )

blank2 = Point()
blank2.x = 1.0
blank2.y = 1.0
dist = distance_between_points(blank, blank2)
dist

1.0

### Rectangles

If we wish to model a rectangle, the first thing that should now be clear is that a rectangle is best modelled with a class (at least in Python). 

Deciding on the exact content of our type is sometimes not straight-forward. How would you implement a rectangle?

Modelling a rectangle can be done in the following ways:

* We can decide to define it with a point and two sides.
* We can choose the point to be a corner or its center.
* We can also use two opposing points.

Here's we'll go with the first option, with the Point being the corner.

In [14]:
class Rectangle:
    """Rectangle model.
    
    Attributes
    ----------
    corner : Point
        bottom left corner
    height : float
    width : float
    """

In [15]:
rect = Rectangle()
rect.width = 100.0
rect.height = 200.0
rect.corner = Point()
rect.corner.x = 0.0
rect.corner.y = 0.0

rect

In [16]:
# We can return instances of classes (just like we do with instances of dictionaries...)
def find_center(rect):
    """ 
    Return a Point to the center of the Rectangle box
    """
    p = Point()
    p.x = rect.corner.x + rect.width / 2
    p.y = rect.corner.y + rect.height / 2
    return p

In [17]:
center = find_center(rect)
print_point(center)

(50.0, 100.0)


In [18]:
# Objects are mutable - we can change their attributes:
    
print(rect.corner.x)
rect.corner.x += 100
print(rect.corner.x)

0.0
100.0


In [19]:
def grow_rectangle(rect, dwidth, dheight):
    """ Take a Rectangle instance and grow it by (dwidth, dheight) """
    rect.width += dwidth
    rect.height += dheight
    # No need to return the instance

In [20]:
rect.width, rect.height

(100.0, 200.0)

In [21]:
grow_rectangle(rect, 100, 100)
rect.width, rect.height

(200.0, 300.0)

### Class Methods

We really haven't done object-oriented programming yet. Our objects currently contain only *data*, in the form of their attributes.

To go one step closer we must learn of _methods_.

Methods are functions bound to objects, describing actions they can do, or that can be done to them. For example, a real-world car can drive. So a `Car` object should have a `drive()` method attached to it. It should also have a `park()` method, and a couple of attributes, like `number_of_wheels`, `manufacturer` and `model`.

As we'll see in a second, the only differentce between methods and functions is that the former are always attached to an object, and they only make sense when attached to that specific object. A `park()` method has no meaning when we try to run it on a `Rectangle()`.

We've already met many methods and used them successfully. For example, we used the `.append(item)` method of a list instance. In this case it's clear why a method is always bound to a specific class - it's irrelevant to "append" an item to an object which is not a list.

Let's add a method to our Point object:

In [63]:
class Point:
    """A 2D point."""
    def transpose(self):
        """Trasnposes by flipping x and y"""
        self.x, self.y = self.y, self.x

In [64]:
p = Point()
p.x = 10
p.y = 20

p.transpose()
print(f"p.x: {p.x}, p.y: {p.y}")

p.x: 20, p.y: 10


The conceptual change here is the following: The active agents here are the *objects*, not the functions. Instead of transpose(point) we have the point "transposing itself": `p.transpose()`.

In [24]:
# The method is a function:
p.transpose
# This returns the address to the function, but doesn't call it, just like regular functions.
# See how it's not a regular function, but a "bound method"?

<bound method Point.transpose of <__main__.Point object at 0x7f6c107966a0>>

In general, most functions that take an instance of some object as one of their parameters should be a candidate for becoming a method, bound to that object, since you might need it later on for other instances as well.

One more technical note here is that even though the added method has `self` as its argument, when we call it we don't need to pass a parameter. `self` acts as a reference to the instance, or object, that we're currently handling. This is what makes methods "special" - they work with the data 'inside' the object they're a part of, and can modify this data if needed. All methods must be defined with the self parameter as their first parameter. Note that `self` isn't a special keyword, rather it's just the convention for the first argument in the method definition.

In [25]:
# This doesn't work, look at the number of arguments:
p.transpose(p)
# Two argumnets were given? We gave only one. The second was the self argument that is implicitly passed.

TypeError: transpose() takes 1 positional argument but 2 were given

One thing is still missing though - each time we create the point we have a three step process:
1. Create the instance: `p = Point()`
2. Add the `x` attribute: `p.x = 2`
3. Add the `y` attribute: `p.y = 3`

First, it would be nice if we could make this process shorter. Second, the Point instance is really unusable unless it has both attributes `(x, y)` set, so we want to make sure that we don't have a Point without both `x` and `y`. This is accomplished by the `__init__` method. 

### The `__init__` method

Classes have several special methods attached to them. While most are out the course's scope, a special method that __does__ deserve a closer look is the `__init__()` method. It has two underscores before and after its name, making it a "dunder" method, short for "double underscore".

The `__init__()` methods allows us to define our class' attributes inside the class definition:

In [66]:
class Point:
    """A 2D point."""
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def transpose(self):
        """Trasnposes by flipping x and y"""
        self.x, self.y = self.y, self.x

Now, in order to create a Point instance, we have to pass in the two arguments that the `__init__` method requires:

In [27]:
p = Point(10, 20)
print(f"p.x: {p.x}, p.y: {p.y}")
p.transpose()
print(f"p.x: {p.x}, p.y: {p.y}")

p.x: 10, p.y: 20
p.x: 20, p.y: 10


As we said, this is better because we enforce our Point user to initialize all attributes, which eases the use of the other methods the Point has. Most chances are that the first method you'll write for a newly defined class is the `__init__()` method. 
Let's look at a broader example using the Rectange we defined earlier and the two other functions we also had.

In [28]:
class Rectangle:
    """Rectangle model.
    
    Attributes
    ----------
    corner : Point
        Bottom left corner
    height, width : float
    """
    def __init__(self, corner, height=10, width=10):
        self.corner = corner
        self.height = height
        self.width = width
        
    def find_center(self):
        """Return a Point to the center of the Rectangle box."""
        x = self.corner.x + self.width / 2
        y = self.corner.y + self.height / 2
        return Point(x, y)

    def grow(self, dwidth, dheight):
        """Change this instance's size by (dwidth, dheight).
        
        Parameters
        ----------
        dwidth, dheight : float
            Change the first and second axes by +dwidth\dheight
        """
        self.width += dwidth
        self.height += dheight

In the example above:
1. The Class name is CamelCase.
2. The docstring of the entire class describes its general purpose.
3. The `__init__` method takes in three arguments, but two of them are optional.
4. We added the two functions we defined earlier to the class as methods, since they only operate on rectangles in the first place.

In [29]:
rect = Rectangle(p)
print(f"rect.width: {rect.width}, rect.height: {rect.height}")

rect.width: 10, rect.height: 10


In [30]:
# Methods can use other methods
class Rectangle:
    """Rectangle model.
    
    Attributes
    ----------
    corner : Point
        Bottom left corner
    height, width : float
    """
    def __init__(self, corner, height=10, width=10):
        self.corner = corner
        self.height = height
        self.width = width
        
    def find_center(self):
        """Return a Point to the center of the Rectangle box."""
        x = self.corner.x + self.width / 2
        y = self.corner.y + self.height / 2
        return Point(x, y)

    def grow(self, dwidth, dheight):
        """Change this instance's size by (dwidth, dheight).
        
        Parameters
        ----------
        dwidth, dheight : float
            Change the first and second axes by +dwidth\dheight
        """
        self.width += dwidth
        self.height += dheight 
    
    def move_to_origin(self):
        """Moves the center of the rectangle to (0, 0)"""
        center = self.find_center()
        if center.x == 0 and center.y == 0:
            return
        self.corner = Point(-self.width/2, -self.height/2)

In [31]:
corner = Point(10, 10)
rect = Rectangle(corner, 4, 4)
center = rect.find_center()
print(f"The center of the rectange is {(center.x, center.y)}")
rect.move_to_origin()
new_center = rect.find_center()
print(f"The center of the moved rectange is {(new_center.x, new_center.y)}")

The center of the rectange is (12.0, 12.0)
The center of the moved rectange is (0.0, 0.0)


Again, you should see how the object modifies itself and acts upon itself using other methods. We're not modifying the internal parts of the instance ourselves - we let the methods do it for us.

#### `__str__`
Another interesting _dunder_ method is the `__str__()` method, which defines what the class will print when invoked using the `print(class_instance)` command. For example:

In [37]:
class ShoppingList:
    def __init__(self, vegetables=10, fruits=5, bread=1):
        self.vegetables = vegetables
        self.fruits = fruits
        self.bread = bread
    
    def __str__(self):
        str_to_print = f"""Shopping List:
            Vegetabels: {self.vegetables}
            Fruits: {self.fruits}
            Bread: {self.bread}
            Total items: {self.vegetables + self.fruits + self.bread}"""
        return str_to_print

In [39]:
slist = ShoppingList()
print(slist)

Shopping List:
            Vegetabels: 10
            Fruits: 5
            Bread: 1
            Total items: 16


In [40]:
shop_list = ShoppingList(5, 1, 3)

# We can change the order of parameters when using keyword arguments
shop_list2 = ShoppingList(fruits=5, bread=1, vegetables=3)

In [45]:
print(f"Fruits: {shop_list.fruits}")
print(f"Shopping list #2:\n{shop_list2}")

Fruits: 1
Shopping list #2:
Shopping List:
            Vegetabels: 3
            Fruits: 5
            Bread: 1
            Total items: 9


The `__str__()` method is very useful for debugging purposes.

### Operator Overloading

One of the most interesting properties of Python is operator overloading (although it's not unique to Python). It means that we can force our self-declared types (i.e. classes) to behave in a certain way with the standard mathematical operations.

We'll use the ShoppingList class as an example. Say we want to __add__ two different shopping lists. Naively, we might just try the following:

In [46]:
shoplist1 = ShoppingList()
shoplist2 = ShoppingList()
print(shoplist1 + shoplist2)

TypeError: unsupported operand type(s) for +: 'ShoppingList' and 'ShoppingList'

To us, this expression seems completely fine - adding two shopping lists should just concatenate the items one after the other. The fact that it's a very readable line of code makes it a _good_ line of code, since you have to remember that we write code for humans to read, not computers.

Unfortunately, Python can't add two shopping lists because it was never taught how to do that. Luckily, we can override the behavior of the addition operator, by defining the `__add__()` method in the class definition:

In [47]:
class ShoppingList:
    ''' Shopping list class
    Attributes:
    vegeteb
    
    '''
    def __init__(self, vegetables=10, fruits=5, bread=1):
        self.vegetables = vegetables
        self.fruits = fruits
        self.bread = bread
    
    def __str__(self):
        str_to_print = f"""Shopping List:
        Vegetabels: {self.vegetables}
        Fruits: {self.fruits}
        Bread: {self.bread}
        Total items: {self.vegetables + self.fruits + self.bread}"""
        return str_to_print
    
    # ----- New method below: ------
    def __add__(self, other):
        """Add together two shopping lists.
        
        Notes
        -----
        This method returns a new shopping list, meaning it doesn't modify
        any of the existing lists it was given.
        """
        new_list = ShoppingList(
            vegetables=self.vegetables + other.vegetables,
            fruits=self.fruits + other.fruits,
            bread=self.bread + other.bread
        )
        return new_list

In [48]:
# Now we can safely add two ShoppingList() instances:
shoplist1 = ShoppingList()
shoplist2 = ShoppingList()
added_shoplist = shoplist1 + shoplist2
print(added_shoplist)

Shopping List:
        Vegetabels: 20
        Fruits: 10
        Bread: 2
        Total items: 32


In [49]:
print(shoplist1)

Shopping List:
        Vegetabels: 10
        Fruits: 5
        Bread: 1
        Total items: 16


In [50]:
# Addition of something other than a ShoppingList will result in an AttributeError:
shoplist1 + 1

# We could write an "if" in the __add__() method that checks if 'other' is an integer, 
# and if so just adds that integer number to all items in the shopping list

AttributeError: 'int' object has no attribute 'vegetables'

We can overload all operators to make our classes behave as one would intuitively expect them to. A quick google query will tell you the name of the dunder method you're after when you wish to overload some operator.

We usually don't write new dunder methods. That is, Python pre-defines the relevant dunder methods for us, and we just re-use their names. Generally you shouldn't implement some random method you thought of as a dunder method.

### Summary

OOP is the most important programming paradigm nowadays, and you should be very familiar with it. Some problems fit this paradigm hand in glove, and the best example is GUI programming. However, it's not the "ultimate" answer to any design difficulty you have. Some problems _can_ be solved by using intricate objects and multiple inheritance, but in reality they're much simpler when solved using procedural design. Remember to write code that humans, and especially your future self, can read and understand.

With that being said, throughout the semester I prefer you write _too many_ objects over writing _too few,_ as it indicates you feel comfortable in the object-oriented world.

## Two Exercises

### The Fraction Class

Create a `Fraction()` class with only the basic attributes. Assume that the inputs are numbers.

a. Override some operators so that you can add a Fraction to an integer (`Fraction() + int` only, not `int + Fraction()`).

b. Override the operator that allows you to check which fraction is larger between the two.

### The Path Class

Create a `Path()` class, symboling a directory in a mock filesystem. The class should at least one attribute, named `files`, containing the list of files in the folder, and three methods, not including the mandatory `__init__` method:
- `Path.get_parent()` - returns the name of the parent folder of our Path (not the entire path). The 
- `Path.get_size()` - returns a number corresponding to the size in KB of the folder, which should depend on the number of files in the folder.
- `Path.set_path(Path())` - Change the current directory to the one given (as a Path() instance).
`

After you've created it, overload the division operator `/` (`__truediv__`) to concatenate a Path class instance with a string pointing to a folder, i.e.:
```python
>> print(Path("/home") / "/usr")
Path("/home/usr")
```
_Note:_ The filesystem is a _mock_ one, meaning that it shouldn't correspond to the actual file system, but to path-like strings you invent, like the ones above.

You can also assume that the string doesn't contain you own path, i.e. you don't have to deal with cases such as:
`Path("/home") / "/home/usr"`

### Exercises solutions follow...

In [51]:
# Exercise 1
class Fraction:
    """Models a fraction with a numerator and a denominator, assuming the inputs are numbers.
    
    Attributes
    ----------
    num : int, float
        numerator
    denom : int, float
        denominator
    value : float 
        num/denom
    
    Notes
    -------
    Can be added and compared to other fractions (overriden __add__ and __gt__)
    """
    def __init__(self, num, denom):
        self.num = num
        self.denom = denom
        if self.denom == 0:
            self.value = None
        else:
            self.value = num / denom
    
    def __str__(self):
        return f"{self.num}/{self.denom} (= {self.value})"
    
    def __add__(self, other):
        """ Left-add a Fraction to an integer """
        if isinstance(other, int):
            num_of_int = self.denom * other
            return Fraction(self.num + num_of_int, self.denom)
    
    def __gt__(self, other):
        """ Left > between two fractions """
        if isinstance(other, Fraction):
            return self.value > other.value
        elif isinstance(other, int):
            return self.value > other 

In [52]:
f1 = Fraction(1, 2)
f2 = Fraction(3, 4)

print(f"f1 = {f1}")
print(f"f2 = {f2}")

print("Is f1 bigger?")
print(f1 > f2)

print("Is f1 smaller?")
print(f1 < f2)

print("Are they equal?")
print(f1 == f2)

print("Are they not equal?")
print(f1 != f2)
# Damn, Python's smart.

f1 = 1/2 (= 0.5)
f2 = 3/4 (= 0.75)
Is f1 bigger?
False
Is f1 smaller?
True
Are they equal?
False
Are they not equal?
True


In [53]:
# Exercise 2 - the Path class
import os


class Path:
    """File system path to folders, allows the use of "/" to move between folders
    
    Attributes
    ----------
    path : str
    files : list of str
    """
    def __init__(self, path, files=['a.txt', 'b.py', 'c.c']):
        self.path = path
        self.files = files
    
    def get_parent(self):
        """ Returns the name of the parent folder """
        separated_path = self.path.split(os.sep)
        if len(separated_path) >= 2:
            return separated_path[-2]
        else:
            return None
    
    def get_size(self):
        """ Returns the size in KB of the folder's content """
        return 10 * len(self.files)
    
    def set_path(self, new_path):
        """ Changes current path to new_path """
        if not isinstance(new_path, Path):
            print("Input variable must be a Path-like instance")
            return
        self.path = new_path.path
        self.files = new_path.files
    
    def __truediv__(self, other):
        """ 
        Traverse the filesystem using the / sign, assuming that 
        other isn't an absolute path.
        Returns a new instance
        """
        assert type(other) in (str, int)
        new_path = self.path + os.sep + str(other)
        new_files = []
        return Path(path=new_path, files=new_files)

In [54]:
p = Path(r'/home/usr/python/python37', files=['ab.R', 'l.cpp', 'py.py'])
print(p.path, p.files)

size = p.get_size()
parent = p.get_parent()
print(f"Size: {size}, parent: {parent}")

/home/usr/python/python37 ['ab.R', 'l.cpp', 'py.py']
Size: 30, parent: python


In [55]:
p.set_path(Path(r'/usr/local/bin'))
print(p.path, p.files)

/usr/local/bin ['a.txt', 'b.py', 'c.c']


In [56]:
new_path = p / 'data'
print(new_path.path)

/usr/local/bin/data
